In [1]:
import os
import sys
import math
import json
import collections
import scipy.stats as sst
import numpy as np
from sklearn import metrics
from tensorflow import keras
sys.path.append('C:\\Users\\Dell\\Desktop\\CV Projects\\ecg')

import warnings
warnings.filterwarnings('ignore')

import load
import network
import utilities

In [2]:
base_dir = 'C:\\Users\\Dell\\Desktop\\CV Projects\\ecg'

In [3]:
model_path = os.path.join(base_dir, 'saved/ecg_experiment_01/090607/0.534-0.954-011-0.430-0.966.hdf5')
data_json = os.path.join(base_dir, 'data/val.json')
config_file = os.path.join(base_dir, 'config.json')

In [4]:
preprocessor = utilities.load(os.path.dirname(model_path))
dataset = load.load_dataset(data_json)

100%|████████████████████████████████████████████████████████████████████████████████| 852/852 [00:28<00:00, 30.23it/s]


In [5]:
preprocessor.int_to_class

{0: 'A', 1: 'N', 2: 'O', 3: '~'}

In [6]:
params = json.load(open(config_file, 'r'))

In [7]:
params.update({
        'input_shape' : [None, 1],
        'num_categories' : len(preprocessor.classes)
    })

In [8]:
model = network.build_network(**params)
model.load_weights(model_path)

In [9]:
#probs = model.predict(ecgs, verbose=0)
#probs.shape

In [10]:
probs = []
labels = []
for x, y  in zip(*dataset):
    x, y = preprocessor.process([x], [y])
    probs.append(model.predict(x))
    labels.append(y)

In [11]:
data_path = os.path.join(base_dir, 'data/train.json')
with open(data_path, 'r') as fid:
    train_labels = [json.loads(l)['labels'] for l in fid]
counts = collections.Counter(preprocessor.class_to_int[l[0]] for l in train_labels)
counts = sorted(counts.most_common(), key=lambda x: x[0])
counts = list(zip(*counts))[1]
smooth = 500
counts = np.array(counts)[None, None, :]
total = np.sum(counts) + counts.shape[1]
prior = (counts + smooth) / float(total)

In [12]:
preds = []
ground_truth = []
for p, g in zip(probs, labels):
    preds.append(sst.mode(np.argmax(p / prior, axis=2).squeeze())[0][0])
    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])

In [ ]:
#preds = []
#ground_truth = []
#for p, g in zip(probs, labels):
#    preds.append(sst.mode(np.argmax(p, axis=2).squeeze())[0][0])
#    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])
#    
#report = classification_report(
#            ground_truth, preds,
#            target_names=preprocessor.classes,
#            digits=3)
#scores = precision_recall_fscore_support(
#                    ground_truth,
#                    preds,
#                    average=None)
#print(report)
#print("Average {:3f}".format(np.mean(scores[2][:3])))

In [13]:
report = metrics.classification_report(
            ground_truth, preds,
            target_names=preprocessor.classes,
            digits=3)
scores = metrics.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                    average=None)
print(report)
print("Average {:3f}".format(np.mean(scores[2][:3])))

              precision    recall  f1-score   support

           A      0.723     0.822     0.769        73
           N      0.916     0.835     0.873       508
           O      0.665     0.759     0.709       228
           ~      0.565     0.605     0.584        43

    accuracy                          0.802       852
   macro avg      0.717     0.755     0.734       852
weighted avg      0.815     0.802     0.806       852

Average 0.783858


In [37]:
def stats(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    stat_dict = {}
    for i in np.unique(y_true):
        tp = np.sum(y_true[y_true==i] == y_pred[y_true==i])
        fp = np.sum(y_true[y_pred==i] != y_pred[y_pred==i])
        fn = np.sum(y_true==i) - tp
        tn = np.sum(y_true!=i) - fp
        stat_dict[i] = (tp, fp, fn, tn)
    return stat_dict


def print_results(sd):
    print('\t\tPrecision  Recall     Specificity     NPV        F1')
    tf1 = 0; tot = 0
    for k, v in sd.items():
        tp, fp, fn, tn = v
        precision = tp / float(tp + fp)
        recall = tp / float(tp + fn)
        specificity = tn / float(tn + fp)
        npv = tn / float(tn + fn)
        f1 = 2 * precision * recall / (precision + recall)
        tf1 += (tp + fn) * f1
        tot += (tp + fn)
        print('{:>10} {:10.3f} {:10.3f} {:10.3f} {:15.3f} {:10.3f}'.format(
            preprocessor.classes[k], precision, recall, specificity, npv, f1))
    print('Average F1 {:.3f}'.format(tf1 / float(tot)))

In [38]:
print_results(stats(ground_truth, preds))

		Precision  Recall     Specificity     NPV        F1
         A      0.723      0.822      0.970           0.983      0.769
         N      0.916      0.835      0.887           0.784      0.873
         O      0.665      0.759      0.861           0.907      0.709
         ~      0.565      0.605      0.975           0.979      0.584
Average F1 0.806
